In [1]:
import json

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

from sklearn.neighbors import KNeighborsClassifier

import json

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
from yandex.get_yandex_ratings import get_items

from sklearn.neighbors import KNeighborsClassifier

import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from matplotlib.markers import MarkerStyle
from postal.parser import parse_address
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.interpolate import spline
from sklearn.cluster import DBSCAN
from collections import Counter
import json



fontsize = 18
figsize = (15,10)
plot_alpha = 0.3
linewidth = 4

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
def str_to_point(s):
    if s is None:
        return None
    l = s.split(' ')
    return {'long': float(l[0]), 'lat': float(l[1])}

In [3]:
train = pd.read_csv('data/train.csv', index_col=0)
test = pd.read_csv('data/test.csv', index_col=0)
test = test.rename(columns={'atm_group': '0-bank'})
train = train.rename(columns={'atm_group': '0-bank'})
banks = np.uint(np.sort(test['0-bank'].unique() * 10))

with open('missed_coords.json', 'r') as infile:
    missed_coords = json.load(infile)

In [4]:
missed_coords = [(str_to_point(item[0]), item[1], item[2], item[3]) for item in missed_coords]

In [5]:
test = test.rename(columns={'lat': '6-lat', 'long': '6-long'})
train = train.rename(columns={'lat': '6-lat', 'long': '6-long'})

# Prepare

In [6]:
test['0-bank'] = np.uint(test['0-bank'] * 10)
train['0-bank'] = np.uint(train['0-bank'] * 10)
columns_for_drop = ['address', 'address_rus']

### Запоминаем индексы

In [7]:
train['is_train'] = True
test['is_train'] = False
test['index'] = test.index + 1
train['index'] = train.index + 1
train['train_index'] = train.index + 1
test['test_index'] = test.index + 1
train['test_index'] = 0
test['train_index'] = 0
train['index'] = train.index

### Создаём общие данные tran-test

In [8]:
data_all = train.append(test, sort=False)

data_all['address_fail'] = 0
data_all.at[data_all['address_rus'].isnull(), 'address_fail'] = 1
data_all['base_target'] = data_all['target']
data_all.at[(data_all['address_fail'] == 1) & (data_all['is_train']), 'target'] = 0

data_all['address_rus'] = data_all['address_rus'].fillna('')

counts = data_all.groupby('address_rus')['id'].count().reset_index().rename(columns={'id':'the_same_address_terminals'})
data_all = pd.merge(data_all, counts, how='left', on='address_rus')
data_all.at[data_all['address_fail'] == 1, 'the_same_address_terminals'] = 0

In [9]:
test['0-bank'] = np.uint(test['0-bank'] * 10)
train['0-bank'] = np.uint(train['0-bank'] * 10)

### Fillna for address_rus

In [10]:
data_all.at[data_all['address_fail'], 'address_rus'] = data_all['address_rus'].fillna("")

### Fillna for lat, long

In [11]:
data_all.at[:, ['6-lat', '6-long']] = data_all[['6-lat', '6-long']].fillna(0)

### Fill missed coords


In [12]:
missed_ids = [item[3] for item in missed_coords]
missed_cities = [item[2] for item in missed_coords]
missed_levels = [1 if item[1] == 'Level.HOUSE' else 0 for item in missed_coords]
missed_lats = [item[0]['lat'] if item[0] is not None else None for item in missed_coords]
missed_longs = [item[0]['long'] if item[0] is not None else None for item in missed_coords]

set_missed_ids = set(missed_ids)
missed_inds = data_all['id'].apply(lambda el: el in set_missed_ids)

In [13]:
data_all.at[missed_inds, '6-lat'] = missed_lats
data_all.at[missed_inds, '6-long'] = missed_longs
data_all['6-levels'] = 1
data_all.at[missed_inds, '6-levels'] = missed_levels
data_all.at[missed_inds, 'city'] = ['' if city is None else city.lower() for city in missed_cities]
data_all['address_fail'] = 0
data_all['address_fail'] = data_all['6-lat'].isnull()
data_all.at[:, ['6-lat', '6-long']] = data_all[['6-lat', '6-long']].fillna(0)

In [14]:
data_all['0-bank_base'] = data_all['0-bank']
data_all = pd.get_dummies(data_all, columns=['0-bank'], prefix='0-bank')

### Детекция городов

In [15]:
def custom_parse_address(row):
    if len(row) == 0:
        return 'NULL'
    d = dict([(item[1], item[0]) for item in parse_address(row)])
    city = ''
    if 'city' in d:
        city = d['city']
    return city

data_all['city-temp'] = data_all['address_rus'].apply(custom_parse_address)

In [16]:
data_all.loc[missed_inds, 'city-temp'] = data_all[missed_inds]['city']
data_all['city'] = data_all['city-temp']
data_all['city'] = list(map(lambda x: x.replace('-', ' ').lower().replace('город ', '').replace(' город', '').replace('ё', 'e'), data_all['city']))
del data_all['city-temp']

In [17]:
# Не задетектились

d = {
    60: 'ст дружба-1',
    299: 'буденновский лес',
    300: 'буденновский лес',
    4443: 'екатеринбург',
    4444: 'екатеринбург',
    4445: 'екатеринбург',
    4446: 'екатеринбург',
    8374: 'усть-илимск',
    8386: 'усть-илимск',
    4609: 'усть-илимск'
}
for k in d:
    data_all.at[k, 'city'] = d[k]
data_all.at[data_all['city'] == '', 'city'] = data_all[data_all['city'] == ''].index.astype(str)

data_all = data_all[['0-bank_base', 'address_rus', 'address', '6-lat', '6-long', 'city']]

In [18]:
data_all.head()

,0-bank_base,address_rus,address,6-lat,6-long,city
0,320,"улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...","EMELYANOVA,34 Y-SAKHALINSK",46.940995,142.738319,южно сахалинск
1,320,"Комсомольская улица, 259, Южно-Сахалинск, Саха...","KOMSOMOLSKAYA,259B Y.SAKHALINSK",46.937353,142.753348,южно сахалинск
2,320,"Коммунистический проспект, Южно-Сахалинск, Сах...","KOMMUN. PR., 32 YUZHNO SAKHAL",46.959413,142.741113,южно сахалинск
3,320,"Ленинградский проспект, 76А, Москва, Россия, 1...","LENINGRADSKIY PR.,76A MOSCOW",55.805827,37.515146,москва
4,320,"Гвардейская площадь, 2, Норильск, Красноярский...","GVARDEYSKAYA PL., 2 NORILSK",69.343541,88.211228,норильск


### Извлечение информации о отделениях банков

In [40]:
def extract_time_schedule(el):
    s = 1000
    f = -1
    weekend = 0
    stop = 0
    if el['type'] == 'atm':
        s, f, weekend = 0, 24, 1
    for sch in ['schedule_general', 'schedule_private_person', 'schedule_entities', 'schedule_vip']:
        stop |= el[sch].find('перерыв') > 0
        s = min([s] + list(map(int, re.findall('\d', el[sch]))))
        f = max([f] + list(map(int, re.findall('\d', el[sch]))))
        if el[sch].find('сб') > -1 or el[sch].find('вс') > -1:
            weekend = 1
    return s, f, weekend, stop

def extract_desc(desc, data, name, debug=0):
    dots = []
    for ind, item in enumerate(desc):
        if item['longitude'] is not None and item['latitude'] is not None and len(item['longitude']) != 0 and len(item['latitude']) != 0:
            dots.append([item['longitude'], item['latitude']])
        else:
            dots.append([1000000, 10000000])

    knc = KNeighborsClassifier()
    knc.fit(X=dots , y=np.ones(len(dots)))

    ds, i_s = knc.kneighbors(X=list(zip(data['6-long'].values, data['6-lat'].values)), n_neighbors=5)

    matched = []
    for ind, item in enumerate(zip(ds, i_s)):
        d, i = item
        if debug == 0:
            print('--------------------------------------------', ind)
            print(d)
            print('BANKI.RU:', desc[i[0]]['longitude'], desc[i[0]]['latitude'], desc[i[0]]['address'])
            print('ADDRESS_RUS:', data['6-long'].iloc[ind], data['6-lat'].iloc[ind], data['address_rus'].iloc[ind])
            print('ADDRESS:', data['address'].iloc[ind])
        args = {
            'active': 0,
            'type': '',
            'time_schedule_s': 0,
            'time_schedule_f': 0,
            'time_schedule_week': 0,
            'time_schedule_stop': 0,
            'metro_name': 0,
            'without_weekend': 0,
            'is_round_the_clock': 0,
            'is_works_as_shop': 0,
            'is_at_closed_place': 0,
            'actual': 0,
            'comment_to_address': '',
            'bank_name': name,
            'index': data.index[ind],
        }
        if d[0] < 5e-3:
            el = desc[i[0]]
            a = extract_time_schedule(el)
            args = {
                'active': int(el['active']),
                'type': el['type'],
                'time_schedule_s': a[0],
                'time_schedule_f': a[1],
                'time_schedule_week': a[2],
                'time_schedule_stop': a[3],
                'metro_name': int(len(el['metro_name']) != 0),
                'without_weekend': el['without_weekend'],
                'is_round_the_clock': int(el['is_round_the_clock']),
                'is_works_as_shop': int(el['is_works_as_shop']),
                'is_at_closed_place': int(el['is_at_closed_place']),
                'actual': 1,
                'comment_to_address': el['comment_to_address'],
                'bank_name': name,
                'index': data.index[ind],
            }
            matched.append(args)
            
            if debug == 1:
                print('--------------------------------------------', ind)
                print(d)
                print('BANKI.RU:', desc[i[0]]['longitude'], desc[i[0]]['latitude'], desc[i[0]]['address'])
                print('ADDRESS_RUS:', data['6-long'].iloc[ind], data['6-lat'].iloc[ind], data['address_rus'].iloc[ind])
                print('ADDRESS:', data['address'].iloc[ind])
            continue
        elif debug == 2:
            print('--------------------------------------------', ind)
            print(d)
            print('BANKI.RU:', desc[i[0]]['longitude'], desc[i[0]]['latitude'], desc[i[0]]['address'])
            print('ADDRESS_RUS:', data['6-long'].iloc[ind], data['6-lat'].iloc[ind], data['address_rus'].iloc[ind])
            print('ADDRESS:', data['address'].iloc[ind])
        matched.append(args)
    return matched

In [41]:
path = 'banki.ru/desc/'
onlyfiles = sorted([f for f in listdir(path) if isfile(join(path, f))])
all_descs = []
for f in tqdm(onlyfiles):
    fd = open(path + f)
    all_descs += json.load(fd)
    fd.close()

100%|██████████| 280/280 [00:02<00:00, 115.20it/s]


In [91]:
banks = {
    'Росбанк': 80830,
    'Ак Барс': 10220,
    'Альфа-Банк': 19420,
    'Газпромбанк': 31855,
    'Райффайзенбанк': 320,
    'Россельхозбанк': 4965,
    'Банк Уралсиб': 54780,
}
bank_dict = {}
for item in all_descs:
    bank_dict[item['bank_name']] = item['bank_id']

In [93]:
ress = []
for i, name in enumerate(banks):
    if i != 6:
        continue
    print(name)
    descs = []
    for item in all_descs:
        if item['bank_name'] == name:
            descs.append(item)
    x = extract_desc(descs, data_all[data_all['0-bank_base'] == banks[name]], name, -1)
    print(len([1 for item in x if item['actual']]) / len(descs))
    ress += x

Банк Уралсиб
0.28440366972477066


/home/hedgehogues/anaconda3/envs/ozon/lib/python3.6/site-packages/sklearn/utils/validation.py:558: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)
/home/hedgehogues/anaconda3/envs/ozon/lib/python3.6/site-packages/sklearn/utils/validation.py:558: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)


In [88]:
answ = {}
for res in ress:
    answ[res['index']] = res
    
pd.concat([pd.DataFrame(answ).T, data_all], axis=1, sort=False)